In [1]:
import pickle
import pandas as pd
import json

## Load test QA

In [2]:
with open('data/qa_on_gost.pkl', 'rb') as f:
    qa = pickle.load(f)
    qa_df = pd.DataFrame(qa)

In [26]:
with open('data/test_qa_with_akela_answer.pickle', 'rb') as f:
    qa = pickle.load(f)
    qa_df = pd.DataFrame(qa)
    qa_df['reranked_paragraphs'] = [[d.page_content for d in r] for r in qa_df['reranked_paragraphs']]

In [3]:
qa_df.to_csv('data/qa_on_gost.csv')

In [33]:
with open('data/otraslevye_docs_gost.json') as f:
    chunks = json.load(f)

In [34]:
len(chunks['ids'])

9394

In [35]:
qa_df['document'] = qa_df['ids'].apply(lambda x: chunks['documents'][chunks['ids'].index(x)])

In [36]:
qa_df.head()

,ids,question,answers,relevance,system_response,akela_answer,reranked_paragraphs,document
0,8f32eb5b-5124-4258-97f1-4e0950adce96,Как правильно ухаживать за электродом измерителя?,Уход за электродом (следуйте инструкциям произ...,8,Для обеспечения точности метода и продления ср...,Уход за электродом измерителя включает следую...,[11.3.8 Установить показания измерителя ма отм...,11.3.15 Выключить измеритель и закрыть его кры...
1,8f32eb5b-5124-4258-97f1-4e0950adce96,"Как восстановить электрод измерителя, если оно...",Восстановление электрода может быть необходимо...,8,"В соответствии с текстом, электрод измерителя ...","Если электрод измерителя стал неисправным, ег...",[11.3.15 Выключить измеритель и закрыть его кр...,11.3.15 Выключить измеритель и закрыть его кры...
2,8f32eb5b-5124-4258-97f1-4e0950adce96,Как проверить калибровку электрода измерителя?,"Проверить отклик электрода, выполнив калибровк...",7,"Согласно тексту, калибровку электрода измерите...",Проверка калибровки электрода измерителя выпо...,[11.3.8 Установить показания измерителя ма отм...,11.3.15 Выключить измеритель и закрыть его кры...
3,0eec8214-7c6f-448e-a9ba-e80c2ec86bb6,Какие изменения в проекте должны быть задокуме...,Все изменения в проекте должны быть задокумент...,8,Все изменения в проекте должны быть задокумент...,Все изменения и модификации в проекте должны ...,[Все изменения в проекте должны быть задокумен...,Все изменения в проекте должны быть задокумент...
4,0eec8214-7c6f-448e-a9ba-e80c2ec86bb6,Какие параметры должны быть приняты во внимани...,Уровни напряженности модифицированных или изме...,7,Основные параметры и размеры фильтров должны с...,При внесении изменений в проект должны быть п...,[6.6.2 При внесении изменений в конструкцию по...,Все изменения в проекте должны быть задокумент...


## Compute metrics

In [37]:
def format_response_text(response_text):
    return response_text.split('<|start_header_id|>assistant<|end_header_id|>\nОтвет:\n')[1]

In [38]:
with open('data/test_qa_responses.json') as f:
    qa_responses = json.load(f)
response_texts = [format_response_text(x['result']) for x in qa_responses]
source_documents = [[d['page_content'] for d in r['source_documents']] for r in qa_responses]

In [39]:
with open('data/test_qa_results_hyde.json') as f:
    qa_responses_hyde = json.load(f)
response_texts_hyde = [format_response_text(x['response']) for x in qa_responses_hyde]
source_documents_hyde = [[d['page_content'] for d in r['source_documents']] for r in qa_responses_hyde]

In [40]:
qa_df['system_response'] = response_texts
qa_df['system_response_hyde'] = response_texts_hyde
qa_df['source_documents'] = source_documents
qa_df['source_documents_hyde'] = source_documents_hyde

In [41]:
def find_hit(ground_truth, candidates):
    if ground_truth in candidates:
        return candidates.index(ground_truth)
    else:
        return None

In [42]:
qa_df.head()

,ids,question,answers,relevance,system_response,akela_answer,reranked_paragraphs,document,system_response_hyde,source_documents,source_documents_hyde
0,8f32eb5b-5124-4258-97f1-4e0950adce96,Как правильно ухаживать за электродом измерителя?,Уход за электродом (следуйте инструкциям произ...,8,Для обеспечения точности метода и продления ср...,Уход за электродом измерителя включает следую...,[11.3.8 Установить показания измерителя ма отм...,11.3.15 Выключить измеритель и закрыть его кры...,"Для обеспечения точности метода, необходимо сл...",[Электроды Для обеспечения точности метода реш...,[Электроды Для обеспечения точности метода реш...
1,8f32eb5b-5124-4258-97f1-4e0950adce96,"Как восстановить электрод измерителя, если оно...",Восстановление электрода может быть необходимо...,8,"В соответствии с текстом, электрод измерителя ...","Если электрод измерителя стал неисправным, ег...",[11.3.15 Выключить измеритель и закрыть его кр...,11.3.15 Выключить измеритель и закрыть его кры...,"Восстановить электрод измерителя можно, если о...",[7.3.1 Проверяют систему «.измерительный прибо...,[7.3.1 Проверяют систему «.измерительный прибо...
2,8f32eb5b-5124-4258-97f1-4e0950adce96,Как проверить калибровку электрода измерителя?,"Проверить отклик электрода, выполнив калибровк...",7,"Согласно тексту, калибровку электрода измерите...",Проверка калибровки электрода измерителя выпо...,[11.3.8 Установить показания измерителя ма отм...,11.3.15 Выключить измеритель и закрыть его кры...,Проверка калибровки электрода измерителя осуще...,[7.3.1 Проверяют систему «.измерительный прибо...,[7.3.1 Проверяют систему «.измерительный прибо...
3,0eec8214-7c6f-448e-a9ba-e80c2ec86bb6,Какие изменения в проекте должны быть задокуме...,Все изменения в проекте должны быть задокумент...,8,Все изменения в проекте должны быть задокумент...,Все изменения и модификации в проекте должны ...,[Все изменения в проекте должны быть задокумен...,Все изменения в проекте должны быть задокумент...,Все изменения в проекте должны быть задокумент...,[Все изменения в проекте должны быть задокумен...,[Все изменения в проекте должны быть задокумен...
4,0eec8214-7c6f-448e-a9ba-e80c2ec86bb6,Какие параметры должны быть приняты во внимани...,Уровни напряженности модифицированных или изме...,7,Основные параметры и размеры фильтров должны с...,При внесении изменений в проект должны быть п...,[6.6.2 При внесении изменений в конструкцию по...,Все изменения в проекте должны быть задокумент...,Уровни напряженности модифицированных или изме...,[Если в процессе производства в конструкцию вн...,[Проектная документация должна включать методы...


In [43]:
qa_df['hitrate'] = qa_df.apply(lambda x: find_hit(x['document'], x['source_documents']), axis=1)
qa_df['hitrate_hyde'] = qa_df.apply(lambda x: find_hit(x['document'], x['source_documents_hyde']), axis=1)
qa_df['hitrate_akela'] = qa_df.apply(lambda x: find_hit(x['document'], x['reranked_paragraphs']), axis=1)

In [44]:
qa_df['hitrate'].value_counts()

0.0    89
1.0    29
2.0    10
Name: hitrate, dtype: int64

In [45]:
qa_df['hitrate_hyde'].value_counts()

0.0    59
1.0    18
2.0    11
Name: hitrate_hyde, dtype: int64

In [46]:
qa_df.notna().sum() / len(qa_df)

ids                      1.000000
question                 1.000000
answers                  1.000000
relevance                1.000000
system_response          1.000000
akela_answer             1.000000
reranked_paragraphs      1.000000
document                 1.000000
system_response_hyde     1.000000
source_documents         1.000000
source_documents_hyde    1.000000
hitrate                  0.533333
hitrate_hyde             0.366667
hitrate_akela            0.233333
dtype: float64

In [49]:
qa_df['reranked_paragraphs'].apply(len).value_counts()

3    234
0      3
2      3
Name: reranked_paragraphs, dtype: int64

In [20]:
qa_df.to_csv('data/test_qa.csv')